In [47]:
pip install haversine


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [48]:
# 위도, 경도 거리 계산 # haversine(A지점, B지점) # default: km
from haversine import haversine

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pickle

In [50]:
# csv 파일 불러오기
customers = pd.read_csv('./dacon_data/open/customers.csv')
locations = pd.read_csv('./dacon_data/open/locations.csv')
orders = pd.read_csv('./dacon_data/open/orders.csv')
order_items = pd.read_csv('./dacon_data/open/order_items.csv')
payments = pd.read_csv('./dacon_data/open/payments.csv')
products = pd.read_csv('./dacon_data/open/products.csv')
reviews = pd.read_csv('./dacon_data/open/reviews.csv')
sellers = pd.read_csv('./dacon_data/open/sellers.csv')

In [51]:
# 통합 데이터 불러오기
with open('./dacon_total_df_20240503_02.pkl', 'rb') as f:
    dacon_total_df_20240503_02 = pickle.load(f)

dacon_total_df_copy = dacon_total_df_20240503_02.copy()
dacon_total_df_copy

,Order_id,Product_id,Seller_id,Order_purchase_year,Order_purchase_month,Review_id,Review_score,Review_creation_date,Review_answer_timestamp,Price,...,Customer_zipcode_prefix,Customer_city,Customer_state,Seller_zipcode_prefix,Seller_city,Seller_state,order_count,Order_purchase_quarter,순이익,매출액
355,ORDER_00329,PRODUCT_07734,SELLER_0146,2017,10,REVIEW_65349,1,2017-10-21,2017-10-22 10:10:28,75.00,...,50090,recife,PE,30190,belo horizonte,MG,1,4,75.00,95.58
666,ORDER_00616,PRODUCT_04278,SELLER_0914,2017,10,REVIEW_21396,1,2017-10-22,2017-10-24 23:33:19,294.90,...,13610,leme,SP,86035,londrina,PR,1,4,294.90,315.24
1321,ORDER_01235,PRODUCT_23369,SELLER_1445,2017,10,REVIEW_50469,3,2017-10-25,2017-10-26 23:52:15,93.90,...,21853,rio de janeiro,RJ,87900,loanda,PR,1,4,93.90,111.51
1385,ORDER_01293,PRODUCT_04377,SELLER_2603,2017,10,REVIEW_75101,1,2017-10-27,2017-10-27 10:41:59,44.90,...,21220,rio de janeiro,RJ,17510,marilia,SP,3,4,134.70,181.59
2230,ORDER_02078,PRODUCT_21531,SELLER_2660,2017,10,REVIEW_58071,4,2017-10-28,2018-01-08 15:09:32,39.90,...,37704,pocos de caldas,MG,2969,sao paulo,SP,2,4,79.80,108.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93368,ORDER_87982,PRODUCT_29370,SELLER_0335,2019,7,REVIEW_24242,4,2019-07-13,2019-07-15 14:23:30,24.90,...,2050,sao paulo,SP,20270,rio de janeiro,RJ,1,3,24.90,40.17
93392,ORDER_88004,PRODUCT_02102,SELLER_1628,2019,7,REVIEW_72610,5,2019-07-28,2019-07-29 02:33:56,70.00,...,82860,curitiba,PR,80330,curitiba,PR,1,3,70.00,83.32
93409,ORDER_88021,PRODUCT_22041,SELLER_1230,2019,7,REVIEW_55262,5,2019-07-26,2019-07-26 22:20:21,40.99,...,13515,charqueada,SP,3275,sao paulo,SP,1,3,40.99,55.59
93424,ORDER_88035,PRODUCT_02524,SELLER_1627,2019,7,REVIEW_32238,5,2019-07-25,2019-07-26 01:32:39,31.50,...,48030,alagoinhas,BA,3006,sao paulo,SP,1,3,31.50,53.65


In [52]:
# locations 좌표 평균값으로 지정 
locations_group_lat=locations.groupby(["Geolocation_zipcode_prefix"])["Geolocation_lat"].mean().reset_index()
locations_group_lng=locations.groupby(["Geolocation_zipcode_prefix"])["Geolocation_lng"].mean().reset_index()
locations_group =  pd.merge(locations_group_lat, locations_group_lng, how = 'inner', on="Geolocation_zipcode_prefix")

In [53]:
# pd.options.display.max_columns = None

In [54]:
# customer 위치 저장
dacon_total_df_copy = pd.merge(dacon_total_df_copy, locations_group, left_on = 'Customer_zipcode_prefix', right_on = 'Geolocation_zipcode_prefix', how = 'inner')
dacon_total_df_copy.rename(columns={'Geolocation_lat':'Customer_lat',"Geolocation_lng" : "Customer_lng" },inplace=True)
dacon_total_df_copy.drop(columns='Geolocation_zipcode_prefix',inplace=True)

In [55]:
# seller 위치 저장
dacon_total_df_copy = pd.merge(dacon_total_df_copy, locations_group, left_on = 'Seller_zipcode_prefix', right_on = 'Geolocation_zipcode_prefix', how = 'inner')
dacon_total_df_copy.rename(columns={'Geolocation_lat':'Seller_lat',"Geolocation_lng" : "Seller_lng" },inplace=True)
dacon_total_df_copy.drop(columns='Geolocation_zipcode_prefix',inplace=True)

In [56]:
# 컬럼 위치 정리
df_1 = dacon_total_df_copy[['Order_id', 'Product_id', 'Seller_id','Review_id', 'Customer_id', 'Customer_unique_id',
                     'Order_purchase_year', 'Order_purchase_month','Order_purchase_quarter', 'order_count', 
                     'Review_score', 'Review_creation_date', 'Review_answer_timestamp', 
                     'Payment_type', 'Payment_installments', 'Product_category_name',
                    'Price','Freight_value', '순이익', '매출액', 'Order_status', 
                    'Order_purchase_timestamp', 'Order_delivered_carrier_date', 'Order_delivered_customer_date', 'Order_estimated_delivery_date',                    
                    'Customer_zipcode_prefix', 'Customer_city', 'Customer_state','Customer_lat', 'Customer_lng',
                    'Seller_zipcode_prefix', 'Seller_city', 'Seller_state', 'Seller_lat', 'Seller_lng']]
df_1

,Order_id,Product_id,Seller_id,Review_id,Customer_id,Customer_unique_id,Order_purchase_year,Order_purchase_month,Order_purchase_quarter,order_count,...,Customer_zipcode_prefix,Customer_city,Customer_state,Customer_lat,Customer_lng,Seller_zipcode_prefix,Seller_city,Seller_state,Seller_lat,Seller_lng
0,ORDER_00329,PRODUCT_07734,SELLER_0146,REVIEW_65349,CUSTOMER_06208,CUSTOMER_Unique_06197,2017,10,4,1,...,50090,recife,PE,-8.074551,-34.895035,30190,belo horizonte,MG,-19.924640,-43.946122
1,ORDER_00616,PRODUCT_04278,SELLER_0914,REVIEW_21396,CUSTOMER_77212,CUSTOMER_Unique_74990,2017,10,4,1,...,13610,leme,SP,-22.189204,-47.388202,86035,londrina,PR,-23.311083,-51.134917
2,ORDER_01235,PRODUCT_23369,SELLER_1445,REVIEW_50469,CUSTOMER_36496,CUSTOMER_Unique_35969,2017,10,4,1,...,21853,rio de janeiro,RJ,-22.857083,-43.486439,87900,loanda,PR,-22.929384,-53.135873
3,ORDER_01293,PRODUCT_04377,SELLER_2603,REVIEW_75101,CUSTOMER_15399,CUSTOMER_Unique_15296,2017,10,4,3,...,21220,rio de janeiro,RJ,-22.849423,-43.313509,17510,marilia,SP,-22.197207,-49.952195
4,ORDER_02078,PRODUCT_21531,SELLER_2660,REVIEW_58071,CUSTOMER_77880,CUSTOMER_Unique_75617,2017,10,4,2,...,37704,pocos de caldas,MG,-21.792839,-46.567365,2969,sao paulo,SP,-23.492059,-46.710628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90188,ORDER_87982,PRODUCT_29370,SELLER_0335,REVIEW_24242,CUSTOMER_61945,CUSTOMER_Unique_60513,2019,7,3,1,...,2050,sao paulo,SP,-23.517344,-46.612135,20270,rio de janeiro,RJ,-22.916245,-43.217950
90189,ORDER_88004,PRODUCT_02102,SELLER_1628,REVIEW_72610,CUSTOMER_52088,CUSTOMER_Unique_51078,2019,7,3,1,...,82860,curitiba,PR,-25.384873,-49.197574,80330,curitiba,PR,-25.469974,-49.307372
90190,ORDER_88021,PRODUCT_22041,SELLER_1230,REVIEW_55262,CUSTOMER_19304,CUSTOMER_Unique_19150,2019,7,3,1,...,13515,charqueada,SP,-22.511338,-47.775884,3275,sao paulo,SP,-23.583987,-46.541397
90191,ORDER_88035,PRODUCT_02524,SELLER_1627,REVIEW_32238,CUSTOMER_15679,CUSTOMER_Unique_15573,2019,7,3,1,...,48030,alagoinhas,BA,-12.137491,-38.405186,3006,sao paulo,SP,-23.541812,-46.624550


In [57]:
# 구매자와 판매자와의 거리 계산 (Km 기준)
dist_list = []
for x in range(len(dacon_total_df_copy)):
    dist = haversine((dacon_total_df_copy.iloc[x]["Customer_lat"],dacon_total_df_copy.iloc[x]["Customer_lng"]),
                      (dacon_total_df_copy.iloc[x]["Seller_lat"], dacon_total_df_copy.iloc[x]["Seller_lng"]))
    dist_list.append(dist)

dist_list

[1638.8219501955803,
 403.92969326631714,
 988.3085672524976,
 685.667522185957,
 189.51597661113095,
 509.40276151204756,
 5.683517685548369,
 338.29862061376303,
 2143.0410913795117,
 517.7141874677953,
 774.9498590809399,
 23.779177559695018,
 2151.9271183549595,
 294.50085897955165,
 461.4584445779128,
 2.7957739051712256,
 84.08819717385904,
 43.289125802013906,
 194.28253491067747,
 202.24709975755908,
 1367.5328093376852,
 2204.7479180415426,
 13.451383838482455,
 132.5617904835518,
 99.84472925327822,
 2643.1710245834306,
 38.317094682112504,
 382.79934744657567,
 382.79934744657567,
 177.49333825592424,
 367.5508576134349,
 875.9980690074062,
 266.6467489741634,
 352.31527591281406,
 356.9580464484995,
 30.277941494377533,
 374.24337990915245,
 6.496228256383015,
 320.1326594498023,
 344.1965381293711,
 279.78673238799627,
 344.4742879173457,
 3.383878181597814,
 394.9379213832799,
 4.876382245483649,
 467.98793008239227,
 2357.427505381671,
 381.3478541463558,
 381.3478541463

In [58]:
dacon_total_df_copy["Distance"] = dist_list

In [59]:
dacon_total_df_copy

,Order_id,Product_id,Seller_id,Order_purchase_year,Order_purchase_month,Review_id,Review_score,Review_creation_date,Review_answer_timestamp,Price,...,Seller_state,order_count,Order_purchase_quarter,순이익,매출액,Customer_lat,Customer_lng,Seller_lat,Seller_lng,Distance
0,ORDER_00329,PRODUCT_07734,SELLER_0146,2017,10,REVIEW_65349,1,2017-10-21,2017-10-22 10:10:28,75.00,...,MG,1,4,75.00,95.58,-8.074551,-34.895035,-19.924640,-43.946122,1638.821950
1,ORDER_00616,PRODUCT_04278,SELLER_0914,2017,10,REVIEW_21396,1,2017-10-22,2017-10-24 23:33:19,294.90,...,PR,1,4,294.90,315.24,-22.189204,-47.388202,-23.311083,-51.134917,403.929693
2,ORDER_01235,PRODUCT_23369,SELLER_1445,2017,10,REVIEW_50469,3,2017-10-25,2017-10-26 23:52:15,93.90,...,PR,1,4,93.90,111.51,-22.857083,-43.486439,-22.929384,-53.135873,988.308567
3,ORDER_01293,PRODUCT_04377,SELLER_2603,2017,10,REVIEW_75101,1,2017-10-27,2017-10-27 10:41:59,44.90,...,SP,3,4,134.70,181.59,-22.849423,-43.313509,-22.197207,-49.952195,685.667522
4,ORDER_02078,PRODUCT_21531,SELLER_2660,2017,10,REVIEW_58071,4,2017-10-28,2018-01-08 15:09:32,39.90,...,SP,2,4,79.80,108.84,-21.792839,-46.567365,-23.492059,-46.710628,189.515977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90188,ORDER_87982,PRODUCT_29370,SELLER_0335,2019,7,REVIEW_24242,4,2019-07-13,2019-07-15 14:23:30,24.90,...,RJ,1,3,24.90,40.17,-23.517344,-46.612135,-22.916245,-43.217950,353.224561
90189,ORDER_88004,PRODUCT_02102,SELLER_1628,2019,7,REVIEW_72610,5,2019-07-28,2019-07-29 02:33:56,70.00,...,PR,1,3,70.00,83.32,-25.384873,-49.197574,-25.469974,-49.307372,14.530102
90190,ORDER_88021,PRODUCT_22041,SELLER_1230,2019,7,REVIEW_55262,5,2019-07-26,2019-07-26 22:20:21,40.99,...,SP,1,3,40.99,55.59,-22.511338,-47.775884,-23.583987,-46.541397,173.724008
90191,ORDER_88035,PRODUCT_02524,SELLER_1627,2019,7,REVIEW_32238,5,2019-07-25,2019-07-26 01:32:39,31.50,...,SP,1,3,31.50,53.65,-12.137491,-38.405186,-23.541812,-46.624550,1536.749517


In [60]:
dacon_total_df_copy['Order_purchase_timestamp']=pd.to_datetime(dacon_total_df_copy['Order_purchase_timestamp'])
dacon_total_df_copy['Order_delivered_carrier_date']=pd.to_datetime(dacon_total_df_copy['Order_delivered_carrier_date'])
dacon_total_df_copy['Order_delivered_customer_date']=pd.to_datetime(dacon_total_df_copy['Order_delivered_customer_date'])
dacon_total_df_copy['Order_estimated_delivery_date']=pd.to_datetime(dacon_total_df_copy['Order_estimated_delivery_date'])

In [61]:
# 물류 처리 시간 - 구매 시간
dacon_total_df_copy["time_diff_purchase_carrier"] = dacon_total_df_copy["Order_delivered_carrier_date"] - dacon_total_df_copy["Order_purchase_timestamp"]
# 실제 배송 날짜 - 구매 시간
dacon_total_df_copy["time_diff_purchase_customer"] = dacon_total_df_copy["Order_delivered_customer_date"] - dacon_total_df_copy["Order_purchase_timestamp"]
# 실제 배송 날짜 - 물류 처리 시간
dacon_total_df_copy["time_diff_carrier_customer"] = dacon_total_df_copy["Order_delivered_customer_date"] - dacon_total_df_copy["Order_delivered_carrier_date"]
# 실제 배송 날짜 - 기대 배송 날짜
dacon_total_df_copy["time_diff_customer_delivery"] = dacon_total_df_copy['Order_estimated_delivery_date'] - dacon_total_df_copy["Order_delivered_customer_date"]

In [62]:
dacon_total_df_copy["time_diff_purchase_carrier_second"] = dacon_total_df_copy["time_diff_purchase_carrier"].apply(lambda x : x.total_seconds())
dacon_total_df_copy["time_diff_purchase_customer_second"] = dacon_total_df_copy["time_diff_purchase_customer"].apply(lambda x : x.total_seconds())
dacon_total_df_copy["time_diff_carrier_customer_second"] = dacon_total_df_copy["time_diff_carrier_customer"].apply(lambda x : x.total_seconds())
dacon_total_df_copy["time_diff_customer_delivery_second"] = dacon_total_df_copy["time_diff_customer_delivery"].apply(lambda x : x.total_seconds())

In [63]:
dacon_total_df_copy['YearMonth'] = dacon_total_df_copy['Order_purchase_timestamp'].dt.strftime('%Y%m') #월별로 분류

In [64]:
with open('./dacon_total_df_240504_00.pkl', 'wb') as f:
    pickle.dump(dacon_total_df_copy,f)

In [65]:
import pickle
with open('./dacon_total_df_240504_00.pkl', 'rb') as f:
    df = pickle.load(f)

In [66]:
dacon_total_df_copy.corr(numeric_only=True).style.background_gradient(cmap='coolwarm', axis=None)

,Order_purchase_year,Order_purchase_month,Review_score,Price,Freight_value,Payment_installments,Customer_zipcode_prefix,Seller_zipcode_prefix,order_count,Order_purchase_quarter,순이익,매출액,Customer_lat,Customer_lng,Seller_lat,Seller_lng,Distance,time_diff_purchase_carrier_second,time_diff_purchase_customer_second,time_diff_carrier_customer_second,time_diff_customer_delivery_second
Order_purchase_year,1.000000,-0.633579,-0.021666,-0.001304,0.026954,-0.057043,-0.034007,-0.010868,0.008005,-0.617778,-0.000584,0.001895,-0.024104,-0.010298,0.019746,0.009780,-0.036696,-0.083258,-0.020885,0.010659,-0.039261
Order_purchase_month,-0.633579,1.000000,0.024299,-0.000211,0.001748,0.037089,-0.000959,-0.015164,-0.003261,0.969899,0.000924,0.000900,0.008274,0.016411,-0.000773,0.000804,0.004365,0.035950,-0.033874,-0.051336,-0.013693
Review_score,-0.021666,0.024299,1.000000,-0.000670,-0.037136,-0.041016,-0.026468,0.024839,-0.084445,0.030925,-0.019222,-0.025070,-0.043951,-0.049495,-0.009685,0.017995,-0.061240,-0.146739,-0.323643,-0.293743,0.261153
Price,-0.001304,-0.000211,-0.000670,1.000000,0.414149,0.303817,0.041890,0.084121,-0.048304,-0.000747,0.939968,0.929968,0.054661,0.013928,0.020639,0.044435,0.082115,0.067602,0.060502,0.038790,0.004059
Freight_value,0.026954,0.001748,-0.037136,0.414149,1.000000,0.202655,0.226660,0.154694,-0.015956,-0.001505,0.394539,0.456822,0.274598,0.090615,0.035594,-0.009463,0.392277,0.090956,0.221030,0.204325,0.038303
Payment_installments,-0.057043,0.037089,-0.041016,0.303817,0.202655,1.000000,0.054158,0.040739,0.038776,0.034653,0.293668,0.298527,0.063610,0.037845,0.010955,-0.008750,0.087184,0.000715,0.048277,0.052314,0.029860
Customer_zipcode_prefix,-0.034007,-0.000959,-0.026468,0.041890,0.226660,0.054158,1.000000,0.071811,0.000193,-0.000555,0.040604,0.056951,0.118085,-0.311187,-0.013437,-0.017983,0.487339,0.009072,0.275057,0.296055,0.060322
Seller_zipcode_prefix,-0.010868,-0.015164,0.024839,0.084121,0.154694,0.040739,0.071811,1.000000,-0.011254,-0.014680,0.080948,0.088749,-0.035933,-0.034553,-0.148113,-0.431209,0.168650,0.006409,0.047138,0.048788,0.120891
order_count,0.008005,-0.003261,-0.084445,-0.048304,-0.015956,0.038776,0.000193,-0.011254,1.000000,-0.003225,0.139360,0.172146,-0.013460,-0.009875,-0.004821,-0.005454,-0.013161,0.042964,-0.007474,-0.025386,0.008476
Order_purchase_quarter,-0.617778,0.969899,0.030925,-0.000747,-0.001505,0.034653,-0.000555,-0.014680,-0.003225,1.000000,0.000439,0.000183,0.007850,0.015757,-0.002798,-0.002349,0.004053,0.038213,-0.047216,-0.066782,-0.005065


In [67]:
df.corr(numeric_only=True).style.background_gradient(cmap='coolwarm', axis=None)

,Order_purchase_year,Order_purchase_month,Review_score,Price,Freight_value,Payment_installments,Customer_zipcode_prefix,Seller_zipcode_prefix,order_count,Order_purchase_quarter,순이익,매출액,Customer_lat,Customer_lng,Seller_lat,Seller_lng,Distance,time_diff_purchase_carrier_second,time_diff_purchase_customer_second,time_diff_carrier_customer_second,time_diff_customer_delivery_second
Order_purchase_year,1.000000,-0.633579,-0.021666,-0.001304,0.026954,-0.057043,-0.034007,-0.010868,0.008005,-0.617778,-0.000584,0.001895,-0.024104,-0.010298,0.019746,0.009780,-0.036696,-0.083258,-0.020885,0.010659,-0.039261
Order_purchase_month,-0.633579,1.000000,0.024299,-0.000211,0.001748,0.037089,-0.000959,-0.015164,-0.003261,0.969899,0.000924,0.000900,0.008274,0.016411,-0.000773,0.000804,0.004365,0.035950,-0.033874,-0.051336,-0.013693
Review_score,-0.021666,0.024299,1.000000,-0.000670,-0.037136,-0.041016,-0.026468,0.024839,-0.084445,0.030925,-0.019222,-0.025070,-0.043951,-0.049495,-0.009685,0.017995,-0.061240,-0.146739,-0.323643,-0.293743,0.261153
Price,-0.001304,-0.000211,-0.000670,1.000000,0.414149,0.303817,0.041890,0.084121,-0.048304,-0.000747,0.939968,0.929968,0.054661,0.013928,0.020639,0.044435,0.082115,0.067602,0.060502,0.038790,0.004059
Freight_value,0.026954,0.001748,-0.037136,0.414149,1.000000,0.202655,0.226660,0.154694,-0.015956,-0.001505,0.394539,0.456822,0.274598,0.090615,0.035594,-0.009463,0.392277,0.090956,0.221030,0.204325,0.038303
Payment_installments,-0.057043,0.037089,-0.041016,0.303817,0.202655,1.000000,0.054158,0.040739,0.038776,0.034653,0.293668,0.298527,0.063610,0.037845,0.010955,-0.008750,0.087184,0.000715,0.048277,0.052314,0.029860
Customer_zipcode_prefix,-0.034007,-0.000959,-0.026468,0.041890,0.226660,0.054158,1.000000,0.071811,0.000193,-0.000555,0.040604,0.056951,0.118085,-0.311187,-0.013437,-0.017983,0.487339,0.009072,0.275057,0.296055,0.060322
Seller_zipcode_prefix,-0.010868,-0.015164,0.024839,0.084121,0.154694,0.040739,0.071811,1.000000,-0.011254,-0.014680,0.080948,0.088749,-0.035933,-0.034553,-0.148113,-0.431209,0.168650,0.006409,0.047138,0.048788,0.120891
order_count,0.008005,-0.003261,-0.084445,-0.048304,-0.015956,0.038776,0.000193,-0.011254,1.000000,-0.003225,0.139360,0.172146,-0.013460,-0.009875,-0.004821,-0.005454,-0.013161,0.042964,-0.007474,-0.025386,0.008476
Order_purchase_quarter,-0.617778,0.969899,0.030925,-0.000747,-0.001505,0.034653,-0.000555,-0.014680,-0.003225,1.000000,0.000439,0.000183,0.007850,0.015757,-0.002798,-0.002349,0.004053,0.038213,-0.047216,-0.066782,-0.005065


In [68]:
variable_importance = {
    # 'Payment_value': 1,
    'Review_score': 0.8,
    'Price': 0.6,
    'time_diff_customer_delivery_second': 0.5,
    'Distance': 0.4,
    'Freight_value': 0.3,
    'time_diff_purchase_customer_second': 0.2,
}

In [69]:
total_importance = sum(variable_importance.values())
total_importance

2.8

In [70]:
normalized_importance = {key: value / total_importance for key, value in variable_importance.items()}

In [71]:
normalized_importance

{'Review_score': 0.28571428571428575,
 'Price': 0.2142857142857143,
 'time_diff_customer_delivery_second': 0.17857142857142858,
 'Distance': 0.14285714285714288,
 'Freight_value': 0.10714285714285715,
 'time_diff_purchase_customer_second': 0.07142857142857144}

In [72]:
dacon_total_df_copy.columns

Index(['Order_id', 'Product_id', 'Seller_id', 'Order_purchase_year',
       'Order_purchase_month', 'Review_id', 'Review_score',
       'Review_creation_date', 'Review_answer_timestamp', 'Price',
       'Freight_value', 'Customer_id', 'Order_status',
       'Order_purchase_timestamp', 'Order_delivered_carrier_date',
       'Order_delivered_customer_date', 'Order_estimated_delivery_date',
       'Payment_type', 'Payment_installments', 'Product_category_name',
       'Customer_unique_id', 'Customer_zipcode_prefix', 'Customer_city',
       'Customer_state', 'Seller_zipcode_prefix', 'Seller_city',
       'Seller_state', 'order_count', 'Order_purchase_quarter', '순이익', '매출액',
       'Customer_lat', 'Customer_lng', 'Seller_lat', 'Seller_lng', 'Distance',
       'time_diff_purchase_carrier', 'time_diff_purchase_customer',
       'time_diff_carrier_customer', 'time_diff_customer_delivery',
       'time_diff_purchase_carrier_second',
       'time_diff_purchase_customer_second',
       'time_diff

In [73]:
head = dacon_total_df_copy.groupby(['Order_id', 'Product_id', 'Seller_id', 'Order_purchase_year', 'Order_purchase_month', 'Review_id', 'Seller_zipcode_prefix']).value_counts().reset_index().sort_values(['order_count','Product_id'], ascending=[False, True])
head

,Order_id,Product_id,Seller_id,Order_purchase_year,Order_purchase_month,Review_id,Seller_zipcode_prefix,Review_score,Review_creation_date,Review_answer_timestamp,...,time_diff_purchase_carrier,time_diff_purchase_customer,time_diff_carrier_customer,time_diff_customer_delivery,time_diff_purchase_carrier_second,time_diff_purchase_customer_second,time_diff_carrier_customer_second,time_diff_customer_delivery_second,YearMonth,count
78812,ORDER_77030,PRODUCT_24966,SELLER_0262,2019,2,REVIEW_51671,12233,1,2019-03-06,2019-03-12 12:46:07,...,7 days 08:47:20,10 days 23:51:46,3 days 15:04:26,2 days 08:37:33,636440.0,949906.0,313466.0,203853.0,201902,1
60177,ORDER_58777,PRODUCT_14642,SELLER_2683,2018,1,REVIEW_58723,89255,5,2018-02-15,2018-02-16 17:14:41,...,1 days 16:49:21,14 days 13:03:21,12 days 20:14:00,20 days 13:11:50,146961.0,1256601.0,1109640.0,1775510.0,201801,1
27461,ORDER_26840,PRODUCT_18605,SELLER_0406,2018,11,REVIEW_82455,15601,5,2018-12-14,2018-12-19 14:14:16,...,4 days 20:11:29,19 days 23:48:43,15 days 03:37:14,5 days 03:40:25,418289.0,1727323.0,1309034.0,445225.0,201811,1
25031,ORDER_24482,PRODUCT_04327,SELLER_1050,2019,2,REVIEW_38838,9291,1,2019-03-02,2019-03-03 00:44:54,...,6 days 06:41:54,8 days 09:01:54,2 days 02:20:00,5 days 03:12:59,542514.0,723714.0,181200.0,443579.0,201902,1
24795,ORDER_24250,PRODUCT_12522,SELLER_0417,2018,12,REVIEW_84898,15025,1,2018-12-29,2018-12-31 12:08:24,...,2 days 04:24:03,14 days 18:44:19,12 days 14:20:16,10 days 14:54:26,188643.0,1277059.0,1088416.0,917666.0,201812,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34086,ORDER_33298,PRODUCT_29468,SELLER_0768,2019,2,REVIEW_86976,19470,4,2019-03-16,2019-03-19 01:04:29,...,3 days 17:17:55,31 days 20:44:35,28 days 03:26:40,-1 days +02:47:28,321475.0,2753075.0,2431600.0,-76352.0,201902,1
38663,ORDER_37778,PRODUCT_29468,SELLER_0768,2019,1,REVIEW_78851,19470,5,2019-01-24,2019-01-25 01:52:42,...,0 days 20:35:03,13 days 01:38:52,12 days 05:03:49,7 days 01:01:12,74103.0,1129132.0,1055029.0,608472.0,201901,1
41344,ORDER_40405,PRODUCT_29468,SELLER_0933,2018,8,REVIEW_79252,8730,5,2018-08-31,2018-09-01 01:10:40,...,0 days 23:37:44,9 days 02:30:43,8 days 02:52:59,13 days 05:51:34,85064.0,786643.0,701579.0,1144294.0,201808,1
45985,ORDER_44961,PRODUCT_29468,SELLER_0933,2019,6,REVIEW_31478,8730,4,2019-06-30,2019-06-30 18:06:34,...,1 days 18:42:17,5 days 00:35:48,3 days 05:53:31,12 days 09:24:29,153737.0,434148.0,280411.0,1070669.0,201906,1


In [74]:
# head["count"].unique()

In [75]:
# head.columns

In [76]:
# head = dacon_total_df_copy.head(30)
# head2 = head.drop(columns=['Order_id', 'Product_id', 'Seller_id', 'Review_id', 'Customer_id', 'Order_status', 'Payment_type', 'Payment_installments', 'Product_category_name', 'Customer_unique_id', 'Customer_zipcode_prefix', 'Customer_city', 'Customer_state', 'Seller_zipcode_prefix', 'Seller_city', 'Seller_state', 'order_count', 'Seller_lat', 'Seller_lng', 'Customer_lat', 'Customer_lng', 'Review_creation_date', 'Review_answer_timestamp', 'Order_purchase_timestamp', 'Order_delivered_carrier_date', 'Order_delivered_customer_date', 'Order_estimated_delivery_date', 'time_diff_purchase_carrier', 'time_diff_purchase_customer', 'time_diff_carrier_customer', 'time_diff_customer_delivery'])
head2 = head.drop(columns=['Order_id', 'Seller_id', 'Review_id', 'Customer_id', 'Order_status', 'Payment_type', 'Payment_installments', 'Product_category_name', 'Customer_unique_id', 'Customer_zipcode_prefix', 'Customer_city', 'Customer_state', 'Seller_zipcode_prefix', 'Seller_city', 'Seller_state', 'order_count', 'Seller_lat', 'Seller_lng', 'Customer_lat', 'Customer_lng', 'Review_creation_date', 'Review_answer_timestamp', 'Order_purchase_timestamp', 'Order_delivered_carrier_date', 'Order_delivered_customer_date', 'Order_estimated_delivery_date', 'time_diff_purchase_carrier', 'time_diff_purchase_customer', 'time_diff_carrier_customer', 'time_diff_customer_delivery'])
head2

,Product_id,Order_purchase_year,Order_purchase_month,Review_score,Price,Freight_value,Order_purchase_quarter,순이익,매출액,Distance,time_diff_purchase_carrier_second,time_diff_purchase_customer_second,time_diff_carrier_customer_second,time_diff_customer_delivery_second,YearMonth,count
78812,PRODUCT_24966,2019,2,1,100.00,10.12,1,2000.00,2202.40,81.001002,636440.0,949906.0,313466.0,203853.0,201902,1
60177,PRODUCT_14642,2018,1,5,51.00,1.20,1,765.00,783.00,1086.439837,146961.0,1256601.0,1109640.0,1775510.0,201801,1
27461,PRODUCT_18605,2018,11,5,65.49,16.22,4,982.35,1225.65,665.601956,418289.0,1727323.0,1309034.0,445225.0,201811,1
25031,PRODUCT_04327,2019,2,1,29.99,7.78,1,419.86,528.78,94.509992,542514.0,723714.0,181200.0,443579.0,201902,1
24795,PRODUCT_12522,2018,12,1,59.00,13.43,4,826.00,1014.02,470.490242,188643.0,1277059.0,1088416.0,917666.0,201812,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34086,PRODUCT_29468,2019,2,4,127.00,34.69,1,127.00,161.69,1875.396881,321475.0,2753075.0,2431600.0,-76352.0,201902,1
38663,PRODUCT_29468,2019,1,5,127.00,12.39,1,127.00,139.39,647.940361,74103.0,1129132.0,1055029.0,608472.0,201901,1
41344,PRODUCT_29468,2018,8,5,127.50,17.14,3,127.50,144.64,308.883363,85064.0,786643.0,701579.0,1144294.0,201808,1
45985,PRODUCT_29468,2019,6,4,127.50,11.91,2,127.50,139.41,20.875922,153737.0,434148.0,280411.0,1070669.0,201906,1


In [77]:
head2[head2['count'] > 1]

,Product_id,Order_purchase_year,Order_purchase_month,Review_score,Price,Freight_value,Order_purchase_quarter,순이익,매출액,Distance,time_diff_purchase_carrier_second,time_diff_purchase_customer_second,time_diff_carrier_customer_second,time_diff_customer_delivery_second,YearMonth,count


In [78]:
head2.drop(columns='count',inplace=True)

In [80]:
head2.head(1)

,Product_id,Order_purchase_year,Order_purchase_month,Review_score,Price,Freight_value,Order_purchase_quarter,순이익,매출액,Distance,time_diff_purchase_carrier_second,time_diff_purchase_customer_second,time_diff_carrier_customer_second,time_diff_customer_delivery_second,YearMonth
78812,PRODUCT_24966,2019,2,1,100.0,10.12,1,2000.0,2202.4,81.001002,636440.0,949906.0,313466.0,203853.0,201902


In [79]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_standardized = scaler.fit_transform(head2)
X_standardized_df = pd.DataFrame(X_standardized, columns=head2.columns)

for column, weight in normalized_importance.items():
    X_standardized_df[column] *= weight

    
weighted_sum = X_standardized_df.sum(axis=1)

ValueError: could not convert string to float: 'PRODUCT_24966'

In [81]:
weighted_sum.max()

NameError: name 'weighted_sum' is not defined

In [ ]:
weighted_sum.min()

In [82]:
dacon_last_id = dacon_total_df_copy['Product_id'].unique()

In [83]:
len(dacon_last_id)

29196

In [84]:
dacon_remake_df = dacon_total_df_copy.copy()

In [85]:
# test = (head2[dacon_total_df_copy['Product_id'] == 'PRODUCT_04327']).drop(["Product_id"],axis=1)
del test

NameError: name 'test' is not defined

In [86]:
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
i =0
for product_id in tqdm(dacon_last_id):
    test_df = head2[head2['Product_id'] == product_id]
    test_df = test_df.drop(["Product_id"],axis=1)
    scaler = StandardScaler()
    X_standardized = scaler.fit_transform(test_df)
    X_standardized_df = pd.DataFrame(X_standardized, columns=test_df.columns)
    for column, weight in normalized_importance.items():
        X_standardized_df[column] *= weight
    weighted_sum = X_standardized_df.sum(axis=1)
    weighted_sum_max=weighted_sum.max()
    max_index = weighted_sum.idxmax()
    
    max_payment_value=test_df.iloc[max_index]["매출액"]
    
    dacon_remake_df.loc[dacon_remake_df["Product_id"]==product_id , "매출액"] = max_payment_value
    
    i += 1
    

100%|█████████████████████████████████████████████████████████████████████████████| 29196/29196 [05:27<00:00, 89.25it/s]


In [87]:
dacon_total_df_copy[dacon_total_df_copy.Product_id.duplicated()]

,Order_id,Product_id,Seller_id,Order_purchase_year,Order_purchase_month,Review_id,Review_score,Review_creation_date,Review_answer_timestamp,Price,...,Distance,time_diff_purchase_carrier,time_diff_purchase_customer,time_diff_carrier_customer,time_diff_customer_delivery,time_diff_purchase_carrier_second,time_diff_purchase_customer_second,time_diff_carrier_customer_second,time_diff_customer_delivery_second,YearMonth
62,ORDER_19141,PRODUCT_01434,SELLER_1696,2017,10,REVIEW_45288,5,2017-11-05,2017-11-07 11:50:28,249.90,...,1842.165076,25 days 02:50:45,30 days 01:57:22,4 days 23:06:37,32 days 09:28:34,2170245.0,2599042.0,428797.0,2798914.0,201710
68,ORDER_21909,PRODUCT_01434,SELLER_1696,2017,10,REVIEW_87178,3,2017-10-26,2017-10-27 23:05:25,249.90,...,302.973741,6 days 03:30:04,12 days 22:10:13,6 days 18:40:09,41 days 12:34:01,531004.0,1116613.0,585609.0,3587641.0,201710
101,ORDER_31750,PRODUCT_09474,SELLER_1696,2017,10,REVIEW_43488,5,2017-10-25,2017-10-26 01:30:00,59.90,...,2117.463295,5 days 19:25:49,6 days 06:27:55,0 days 11:02:06,57 days 18:57:54,501949.0,541675.0,39726.0,4993074.0,201710
105,ORDER_33077,PRODUCT_20195,SELLER_1696,2017,10,REVIEW_32198,5,2017-10-21,2017-10-22 04:09:29,339.90,...,905.079613,4 days 01:32:11,7 days 01:32:11,3 days 00:00:00,51 days 10:37:47,351131.0,610331.0,259200.0,4444667.0,201710
107,ORDER_34318,PRODUCT_03334,SELLER_1546,2017,10,REVIEW_38486,3,2017-11-25,2017-11-26 21:22:37,59.90,...,1082.619121,35 days 00:59:38,45 days 03:16:37,10 days 02:16:59,13 days 09:04:17,3027578.0,3899797.0,872219.0,1155857.0,201710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90188,ORDER_87982,PRODUCT_29370,SELLER_0335,2019,7,REVIEW_24242,4,2019-07-13,2019-07-15 14:23:30,24.90,...,353.224561,2 days 21:39:21,6 days 06:30:13,3 days 08:50:52,12 days 06:11:08,250761.0,541813.0,291052.0,1059068.0,201907
90189,ORDER_88004,PRODUCT_02102,SELLER_1628,2019,7,REVIEW_72610,5,2019-07-28,2019-07-29 02:33:56,70.00,...,14.530102,1 days 13:29:56,3 days 01:05:13,1 days 11:35:17,4 days 00:56:43,134996.0,263113.0,128117.0,349003.0,201907
90190,ORDER_88021,PRODUCT_22041,SELLER_1230,2019,7,REVIEW_55262,5,2019-07-26,2019-07-26 22:20:21,40.99,...,173.724008,2 days 15:11:54,4 days 16:36:34,2 days 01:24:40,11 days 06:22:20,227514.0,405394.0,177880.0,973340.0,201907
90191,ORDER_88035,PRODUCT_02524,SELLER_1627,2019,7,REVIEW_32238,5,2019-07-25,2019-07-26 01:32:39,31.50,...,1536.749517,1 days 12:45:44,9 days 23:01:43,8 days 10:15:59,14 days 04:41:01,132344.0,860503.0,728159.0,1226461.0,201907


In [88]:
head5_1 = dacon_total_df_copy[dacon_total_df_copy.Product_id=="PRODUCT_29370"].sort_values('순이익', ascending=False).head(5)

In [89]:
head5_1[['Product_id','Seller_id','순이익','매출액']]

,Product_id,Seller_id,순이익,매출액
86070,PRODUCT_29370,SELLER_0335,24.9,43.98
87590,PRODUCT_29370,SELLER_0335,24.9,40.17
76910,PRODUCT_29370,SELLER_0335,24.9,40.13
80772,PRODUCT_29370,SELLER_0335,24.9,32.45
81664,PRODUCT_29370,SELLER_0335,24.9,43.13


In [90]:
head5_2 = dacon_remake_df[dacon_remake_df.Product_id=="PRODUCT_29370"].sort_values('순이익', ascending=False).head(5)

In [91]:
head5_2[['Product_id','Seller_id','순이익','매출액']]

,Product_id,Seller_id,순이익,매출액
86070,PRODUCT_29370,SELLER_0335,24.9,88.33
87590,PRODUCT_29370,SELLER_0335,24.9,88.33
76910,PRODUCT_29370,SELLER_0335,24.9,88.33
80772,PRODUCT_29370,SELLER_0335,24.9,88.33
81664,PRODUCT_29370,SELLER_0335,24.9,88.33


In [92]:
dacon_total_df_copy.매출액.sum()

13854366.929999998

In [93]:
dacon_remake_df.매출액.sum() 

19804085.93

In [94]:
dacon_remake_df.매출액.sum()  - dacon_total_df_copy.매출액.sum()

5949719.000000002

In [95]:
dacon_total_df_copy.corr(numeric_only=True).style.background_gradient(cmap='coolwarm', axis=None)

,Order_purchase_year,Order_purchase_month,Review_score,Price,Freight_value,Payment_installments,Customer_zipcode_prefix,Seller_zipcode_prefix,order_count,Order_purchase_quarter,순이익,매출액,Customer_lat,Customer_lng,Seller_lat,Seller_lng,Distance,time_diff_purchase_carrier_second,time_diff_purchase_customer_second,time_diff_carrier_customer_second,time_diff_customer_delivery_second
Order_purchase_year,1.000000,-0.633579,-0.021666,-0.001304,0.026954,-0.057043,-0.034007,-0.010868,0.008005,-0.617778,-0.000584,0.001895,-0.024104,-0.010298,0.019746,0.009780,-0.036696,-0.083258,-0.020885,0.010659,-0.039261
Order_purchase_month,-0.633579,1.000000,0.024299,-0.000211,0.001748,0.037089,-0.000959,-0.015164,-0.003261,0.969899,0.000924,0.000900,0.008274,0.016411,-0.000773,0.000804,0.004365,0.035950,-0.033874,-0.051336,-0.013693
Review_score,-0.021666,0.024299,1.000000,-0.000670,-0.037136,-0.041016,-0.026468,0.024839,-0.084445,0.030925,-0.019222,-0.025070,-0.043951,-0.049495,-0.009685,0.017995,-0.061240,-0.146739,-0.323643,-0.293743,0.261153
Price,-0.001304,-0.000211,-0.000670,1.000000,0.414149,0.303817,0.041890,0.084121,-0.048304,-0.000747,0.939968,0.929968,0.054661,0.013928,0.020639,0.044435,0.082115,0.067602,0.060502,0.038790,0.004059
Freight_value,0.026954,0.001748,-0.037136,0.414149,1.000000,0.202655,0.226660,0.154694,-0.015956,-0.001505,0.394539,0.456822,0.274598,0.090615,0.035594,-0.009463,0.392277,0.090956,0.221030,0.204325,0.038303
Payment_installments,-0.057043,0.037089,-0.041016,0.303817,0.202655,1.000000,0.054158,0.040739,0.038776,0.034653,0.293668,0.298527,0.063610,0.037845,0.010955,-0.008750,0.087184,0.000715,0.048277,0.052314,0.029860
Customer_zipcode_prefix,-0.034007,-0.000959,-0.026468,0.041890,0.226660,0.054158,1.000000,0.071811,0.000193,-0.000555,0.040604,0.056951,0.118085,-0.311187,-0.013437,-0.017983,0.487339,0.009072,0.275057,0.296055,0.060322
Seller_zipcode_prefix,-0.010868,-0.015164,0.024839,0.084121,0.154694,0.040739,0.071811,1.000000,-0.011254,-0.014680,0.080948,0.088749,-0.035933,-0.034553,-0.148113,-0.431209,0.168650,0.006409,0.047138,0.048788,0.120891
order_count,0.008005,-0.003261,-0.084445,-0.048304,-0.015956,0.038776,0.000193,-0.011254,1.000000,-0.003225,0.139360,0.172146,-0.013460,-0.009875,-0.004821,-0.005454,-0.013161,0.042964,-0.007474,-0.025386,0.008476
Order_purchase_quarter,-0.617778,0.969899,0.030925,-0.000747,-0.001505,0.034653,-0.000555,-0.014680,-0.003225,1.000000,0.000439,0.000183,0.007850,0.015757,-0.002798,-0.002349,0.004053,0.038213,-0.047216,-0.066782,-0.005065


In [ ]:
dacon_remake_df.corr(numeric_only=True).style.background_gradient(cmap='coolwarm', axis=None)